# Countdown Problem 

You are given 6 random integers and a target integer.  Develop a program that constructs an equation that evaluates to the requested target.  You can use each number only once.  The allowed operators are ``+, -, *, /``, and parentheses ``( )``, which you can use as many times as you like.

Additionally, there are rules for any intermediate step of the equation:

*  Cannot reach a negative value, e.g., ``2 - (1 - 3) = 4`` is not an allowed solution, whereas ``2 + (3 - 1)`` is acceptable.
*  Cannot reach a non-integer value, e.g., ``(1 / 2) * 6 = 3`` is not an allowed solution, whereas ``6 - 1 - 2`` is acceptable.

For example:

Input:
numbers = ``[3, 2, 10, 1, 7, 9]``
target = ``478``

Output:
``(10 * 8 * 6) - ((7 + 1) - (9 - 3))``

Note: There are a few more solutions, such as ``(2 + ((10 + 7) * (1 + (3 * 9))))``.

If you really want to test your program, here is a tricky example:
numbers = ``[1, 2, 3, 4, 5, 6]``
target = ``279``

Output:
``((2 + (3 + 4)) * (1 + (5 * 6)))``

# Solution

## Solution: Overview

The solution to this problem involves a brute-force approach.

We aim to generate all possible and valid expressions and evaluate them one after another until we reach our target.

To accomplish this, we encapsulate the concept of an expression using two classes. The key observation is that an expression can be either a single value or two expressions joined by an operator. We model them using two classes:

*  ``ExpressionValue``
*  ``Expression``

Next, we build generators to create all possible and valid expressions. Here are the steps:

*  Generate all permutations of the input numbers.
*  For each permutation, generate splits. For example, the permutation ``[1, 2, 3]`` can be split into ``[([1], [2, 3]), ([1, 2], [3])]``.
*  For each split, generate all possible expressions recursively.
    *  For ``[1]``, we only have one possible expression: ``1``.
    *  For ``[2, 3]``, we have four possible expressions: ``[(2+3), (2-3), (2*3), (2/3)]``.  Due to additional rules, some of these will be invalid, and we might end up with, for example, ``[(2+3), (2*3)]``.
    *  If the list has more than two elements, we generate further splits recursively.
*  Combine expressions from the two parts of the split using all available operators, ensuring the resulting expressions are valid.
    *  Expressions for ``[1]`` are ``[1]``.
    *  Expressions for ``[2, 3]`` are ``[(2+3), (2*3)]``.
    *  We generate combinations such as ``[(1+(2+3)), (1+(2*3)), (1-(2+3)), (1-(2*3)), (1*(2+3)), (1*(2*3)), (1/(2+3)), (1/(2*3))]``. Some of these will be invalid, and we might end up with ``[(1+(2+3)), (1+(2*3)), (1(2+3)), (1*(2*3))]``.

Once the generators are ready, we loop through the results until we find an expression that evaluates to the requested target.

## Solution: Technical note

In this implementation, we rely on generators rather than lists. This is done on purpose to reduce memory footprint, which, in the case of 6 numbers and 4 operators, would take a significant amount of memory and could cause your browser to crash.

The solution is accompanied by unit tests, located towards the bottom of this script.

## Solution: Imports 

In [469]:
from enum import Enum
import itertools
import unittest

## Solution: Define data structures 

An expression (=equation) is encapsulated in 2 classes:

* ExpressionValue which simply is a value
* Expression which is made up up two Expressions and an operator.  Child expressions can be either of type Expression or ExpressionValue.

Because of Python's nature, there's only implicit common interface for ExpressionValue and Expression classes.  In stringly-typed language we'd have common interface that both classes would have to implement. 

In [470]:
class EnumOperator(Enum):
    ADD = 1
    SUBTRACT = 2 
    MULTIPLY = 3 
    DIVIDE = 4
class ExpressionValue:
    def __init__(self, int_value):
        self.int_value = int_value
    def to_string(self):
        return str(self.int_value)
    def evaluate(self):
        return self.int_value
    def validate(self):
        return True 
class Expression:
    def __init__(self, left_expression, right_expression, enum_operator):
        self.left_expression = left_expression 
        self.right_expression = right_expression 
        self.enum_operator = enum_operator
    def to_string(self):
        str_operator = None 
        match self.enum_operator:
            case EnumOperator.ADD: 
                str_operator =  "+"
            case EnumOperator.SUBTRACT: 
                str_operator =  "-"
            case EnumOperator.MULTIPLY: 
                str_operator =  "*"
            case EnumOperator.DIVIDE: 
                str_operator =  "/"
        return "(" +  self.left_expression.to_string() + str_operator + self.right_expression.to_string() + ")"
    def evaluate(self):
        match self.enum_operator:
            case EnumOperator.ADD: 
                return self.left_expression.evaluate() + self.right_expression.evaluate()
            case EnumOperator.SUBTRACT: 
                return self.left_expression.evaluate() - self.right_expression.evaluate() 
            case EnumOperator.MULTIPLY: 
                return self.left_expression.evaluate() * self.right_expression.evaluate() 
            case EnumOperator.DIVIDE: 
                return self.left_expression.evaluate() / self.right_expression.evaluate() 
    def validate(self):
        match self.enum_operator:
            case EnumOperator.SUBTRACT:
                return self.evaluate() >= 0
            case EnumOperator.DIVIDE:
                try:
                    value = self.evaluate()
                    return value % 1 == 0
                except:
                    return False 
            case _:
                return True

## Solution: Functions

In [481]:
def generate_all_permutations(arr_numbers):
    return list(itertools.permutations(arr_numbers))
    
def generate_all_splits(arr_numbers):
    list_return = [] 
    if (len(arr_numbers) <= 2):
        list_return.append(arr_numbers)
    else:
        for i in range(1, len(arr_numbers)):
            list_return.append((arr_numbers[:i], arr_numbers[i:]))
    return list_return
    
def generate_all_expressions(list_numbers, list_operators):
    list_all_permutations = generate_all_permutations(list_numbers)
    for list_permutation in list_all_permutations:
        yield generate_expressions(list_permutation, list_operators)

def generate_expressions(list_numbers, list_operators):
    if (len(list_numbers) == 1):
        yield ExpressionValue(list_numbers[0])
    elif (len(list_numbers) == 2):
        expression_left = ExpressionValue(list_numbers[0])
        expression_right = ExpressionValue(list_numbers[1])
        for operator in list_operators:
            expression = Expression(expression_left, expression_right, operator)
            if expression.validate():
                yield expression
    else:
        list_splits = generate_all_splits(list_numbers)
        for tuple_split in list_splits:
            list_expressions_left = generate_expressions(tuple_split[0], list_operators)
            list_expressions_right = generate_expressions(tuple_split[1], list_operators)
            for expression_left in list_expressions_left:
                for expression_right in list_expressions_right:
                    for operator in list_operators:
                        expression = Expression(expression_left, expression_right, operator)
                        if expression.validate():
                            yield expression
                            
def find_expression(list_numbers, list_operators, int_target):
    list_expression_generators = generate_all_expressions(list_numbers, list_operators)
    for expression_level_1 in list_expression_generators:
        for expression_level_2 in expression_level_1:
            if expression_level_2.evaluate() == int_target:
                return expression_level_2
    return None

# Test

Enter:

*  List of numbers
*  List of operators
*  Target

And validate that returned expression is correct.

In [480]:
#list_test_numbers = [3, 2, 10, 1, 7, 9]
list_test_numbers = [1, 2, 3, 4, 5, 6]
list_test_operators = [EnumOperator.ADD, EnumOperator.SUBTRACT, EnumOperator.MULTIPLY, EnumOperator.DIVIDE]
#int_test_target = 478
int_test_target = 279
expression = find_expression(list_test_numbers, list_test_operators, int_test_target)
if expression:
    display(expression.to_string())
else:
    display("No results")

'((2+(3+4))*(1+(5*6)))'

# Unit Tests

In [473]:
class ValidateExpression(unittest.TestCase):
    def test_validate_value_expression_always_true(self):
        # Arrange 
        expression = ExpressionValue(1)
        # Act 
        bool_is_valid = expression.validate()
        # Assert 
        self.assertTrue(bool_is_valid)
    def test_validate_subtract_negative_value(self):
        # Arrange
        expression = Expression(ExpressionValue(1), ExpressionValue(2), EnumOperator.SUBTRACT) 
        # Act 
        bool_is_valid = expression.validate()
        # Assert 
        self.assertFalse(bool_is_valid)
    def test_validate_subtract_positive_value(self):
        # Arrange
        expression = Expression(ExpressionValue(2), ExpressionValue(1), EnumOperator.SUBTRACT) 
        # Act 
        bool_is_valid = expression.validate()
        # Assert 
        self.assertTrue(bool_is_valid)
    def test_validate_divide_non_integer_value(self):
        # Arrange
        expression = Expression(ExpressionValue(1), ExpressionValue(2), EnumOperator.DIVIDE) 
        # Act 
        bool_is_valid = expression.validate()
        # Assert 
        self.assertFalse(bool_is_valid)
    def test_validate_divide_positive_value(self):
        # Arrange
        expression = Expression(ExpressionValue(4), ExpressionValue(2), EnumOperator.DIVIDE) 
        # Act 
        bool_is_valid = expression.validate()
        # Assert 
        self.assertTrue(bool_is_valid)
    def test_validate_divide_by_zero(self):
        # Arrange 
        expression_value_1 = ExpressionValue(1)
        expression_2 = Expression(ExpressionValue(8), ExpressionValue(4), EnumOperator.DIVIDE)
        expression_value_2 = ExpressionValue(2)
        expression = Expression(expression_value_1, Expression(expression_2, expression_value_2, EnumOperator.SUBTRACT), EnumOperator.DIVIDE)
        # Act
        bool_is_valid = expression.validate()
        # Assert 
        self.assertFalse(bool_is_valid)

In [474]:
class FindEquation(unittest.TestCase):
    def test_find_expression_when_solution_exists(self):
        # Arrange 
        list_numbers = [1, 2, 3]
        list_operators = [EnumOperator.ADD, EnumOperator.SUBTRACT]
        int_target = 4
        # Act 
        expression = find_expression(list_numbers, list_operators, int_target)
        int_expression_value = expression.evaluate()
        # Assert 
        self.assertIsNotNone(expression)
        self.assertEqual(int_expression_value, int_target)
    def test_find_expression_when_solution_doesnt_exist(self):
        # Arrange 
        list_numbers = [1, 2, 3]
        list_operators = [EnumOperator.ADD, EnumOperator.SUBTRACT]
        int_target = 7
        # Act 
        expression = find_expression(list_numbers, list_operators, int_target)
        # Assert 
        self.assertIsNone(expression)

In [475]:
class GeneratePermutations(unittest.TestCase):
    def test_generate_all_permutations_2_elements(self):
        # Arrange 
        arr_numbers = [1, 2]
        int_expected_permutations_count = 2
        # Act 
        list_permutations = generate_all_permutations(arr_numbers)
        # Assert 
        self.assertEqual(len(list_permutations), int_expected_permutations_count)
    def test_generate_all_permutations_3_elements(self):
        # Arrange 
        arr_numbers = [1, 2, 3]
        int_expected_permutations_count = 6
        # Act 
        list_permutations = generate_all_permutations(arr_numbers)
        # Assert 
        self.assertEqual(len(list_permutations), int_expected_permutations_count)
    def test_generate_all_splits_2_elements(self):
        # Arange 
        arr_numbers = [1, 2]
        int_expected_split_count = 1
        # Act 
        list_splits = generate_all_splits(arr_numbers)
        # Assert 
        self.assertEqual(len(list_splits), int_expected_split_count)
    def test_generate_all_splits_3_elements(self):
        # Arange 
        arr_numbers = [1, 2, 3]
        int_expected_split_count = 2
        # Act 
        list_splits = generate_all_splits(arr_numbers)
        # Assert 
        self.assertEqual(len(list_splits), int_expected_split_count)
    def test_generate_all_splits_4_elements(self):
        # Arange 
        arr_numbers = [1, 2, 3, 4]
        int_expected_split_count = 3
        # Act 
        list_splits = generate_all_splits(arr_numbers)
        # Assert 
        self.assertEqual(len(list_splits), int_expected_split_count)

In [476]:
class GenerateExpressions(unittest.TestCase):
    def test_generate_all_expressions_returns_only_valid_expressions(self):
        # Arrange 
        list_numbers = [1, 2]
        list_operators = [EnumOperator.SUBTRACT]
        int_expected_number_of_expressions = 1 
        str_expected_expression = "(2-1)"
        # Act 
        list_expression_generators = generate_all_expressions(list_numbers, list_operators)
        list_expressions = list(itertools.chain(*list_expression_generators))
        # Assert 
        self.assertEqual(len(list_expressions), int_expected_number_of_expressions)
        self.assertEqual(list_expressions[0].to_string(), str_expected_expression)
    def test_generate_all_expressions_1_numbers_1_operator(self):
        # Arrange 
        list_numbers = [1]
        list_operators = [EnumOperator.ADD]
        int_expected_number_of_expressions = 1
        # Act 
        list_expression_generators = generate_all_expressions(list_numbers, list_operators)
        #list_expressions = list(list_expression_generators)
        list_expressions = list(itertools.chain(*list_expression_generators))
        # Assert 
        self.assertEqual(len(list_expressions), int_expected_number_of_expressions)
    def test_generate_all_expressions_2_numbers_1_operator(self):
        # Arrange 
        list_numbers = [1, 2]
        list_operators = [EnumOperator.ADD]
        int_expected_number_of_expressions = 2
        # (1+2)
        # (2+1)
        # Act 
        list_expression_generators = generate_all_expressions(list_numbers, list_operators)
        list_expressions = list(itertools.chain(*list_expression_generators))
        # Assert 
        self.assertEqual(len(list_expressions), int_expected_number_of_expressions)
    def test_generate_all_expressions_2_numbers_2_operator(self):
        # Arrange 
        list_numbers = [1, 2]
        list_operators = [EnumOperator.ADD, EnumOperator.MULTIPLY]
        int_expected_number_of_expressions = 4
        # (1+2) (1*2)
        # (2+1) (2*1)
        # Act 
        list_expression_generators = generate_all_expressions(list_numbers, list_operators)
        list_expressions = list(itertools.chain(*list_expression_generators))
        # Assert 
        self.assertEqual(len(list_expressions), int_expected_number_of_expressions)
    def test_generate_all_expressions_3_numbers_1_operator(self):
        # Arrange 
        list_numbers = [1, 2, 3]
        list_operators = [EnumOperator.ADD]
        int_expected_number_of_expressions = 12
        #  1: (1+2)+3
        #  2: 1+(2+3)
        #  3: (1+3)+2
        #  4: 1+(3+2)
        #  5: (2+1)+3
        #  6: 2+(1+3)
        #  7: (2+3)+1
        #  8: 2+(3+1)
        #  9: (3+1)+2
        # 10: 3+(1+2)
        # 11: (3+2)+1
        # 12: 3+(2+1)
        
        # Act 
        list_expression_generators = generate_all_expressions(list_numbers, list_operators)
        list_expressions = list(itertools.chain(*list_expression_generators))
        # Assert 
        self.assertEqual(len(list_expressions), int_expected_number_of_expressions)

In [479]:
class ExpressionTestCase(unittest.TestCase):
    #
    # Testing operations, that they return the correct results.
    #
    def test_evaluate_add(self):
        # Arrange 
        exp = Expression(ExpressionValue(1), ExpressionValue(2), EnumOperator.ADD)
        int_expected_value = 3
        # Act 
        int_return_value = exp.evaluate()
        # Assert 
        self.assertEqual(int_expected_value, int_return_value)
    def test_evaluate_subtract(self):
        # Arrange 
        exp = Expression(ExpressionValue(3), ExpressionValue(1), EnumOperator.SUBTRACT)
        int_expected_value = 2
        # Act 
        int_return_value = exp.evaluate()
        # Assert 
        self.assertEqual(int_expected_value, int_return_value)
    def test_evaluate_multiply(self):
        # Arrange 
        exp = Expression(ExpressionValue(3), ExpressionValue(2), EnumOperator.MULTIPLY)
        int_expected_value = 6
        # Act 
        int_return_value = exp.evaluate()
        # Assert 
        self.assertEqual(int_expected_value, int_return_value)
    def test_evaluate_divide(self):
        # Arrange 
        exp = Expression(ExpressionValue(8), ExpressionValue(4), EnumOperator.DIVIDE)
        int_expected_value = 2
        # Act 
        int_return_value = exp.evaluate()
        # Assert 
        self.assertEqual(int_expected_value, int_return_value)
    def test_evaluate_value(self):
        # Arrange 
        exp = ExpressionValue(1)
        int_expected_value = 1
        # Act 
        int_return_value = exp.evaluate()
        # Assert 
        self.assertEqual(int_expected_value, int_return_value)
    #
    # Test to_string.
    #
    def test_to_string_add(self):
        # Arrange 
        exp = Expression(ExpressionValue(1), ExpressionValue(2), EnumOperator.ADD)
        str_expected = "(1+2)"
        # Act 
        str_return = exp.to_string()
        # Assert 
        self.assertEqual(str_expected, str_return)
    def test_to_string_subtract(self):
        # Arrange 
        exp = Expression(ExpressionValue(3), ExpressionValue(1), EnumOperator.SUBTRACT)
        str_expected = "(3-1)"
        # Act 
        str_return = exp.to_string()
        # Assert 
        self.assertEqual(str_expected, str_return)
    def test_to_string_multiply(self):
        # Arrange 
        exp = Expression(ExpressionValue(3), ExpressionValue(2), EnumOperator.MULTIPLY)
        str_expected = "(3*2)"
        # Act 
        str_return = exp.to_string()
        # Assert 
        self.assertEqual(str_expected, str_return)
    def test_to_string_divide(self):
        # Arrange 
        exp = Expression(ExpressionValue(8), ExpressionValue(4), EnumOperator.DIVIDE)
        str_expected = "(8/4)"
        # Act 
        str_return = exp.to_string()
        # Assert 
        self.assertEqual(str_expected, str_return)
    def test_to_string_value_expression(self):
        # Arrange 
        exp = ExpressionValue(1)
        str_expected = "1"
        # Act 
        str_return = exp.to_string()
        # Assert 
        self.assertEqual(str_expected, str_return)
    #
    # Test complex expressions
    #
    def test_evaluate_complex_expression(self):
        # Arrange 
        exp_inner = Expression(ExpressionValue(1), ExpressionValue(2), EnumOperator.ADD)
        exp_outer = Expression(exp_inner, ExpressionValue(3), EnumOperator.MULTIPLY)
        str_expected = 9
        # Act 
        str_return = exp_outer.evaluate()
        # Assert 
        self.assertEqual(str_expected, str_return)
    def test_to_string_complex_expression(self):
        # Arrange 
        exp_inner = Expression(ExpressionValue(1), ExpressionValue(2), EnumOperator.ADD)
        exp_outer = Expression(exp_inner, ExpressionValue(3), EnumOperator.MULTIPLY)
        str_expected = "((1+2)*3)"
        # Act 
        str_return = exp_outer.to_string()
        # Assert 
        self.assertEqual(str_expected, str_return)

## Unit Tests: Runner (All)

In [478]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_evaluate_add (__main__.ExpressionTestCase) ... ok
test_evaluate_complex_expression (__main__.ExpressionTestCase) ... ok
test_evaluate_divide (__main__.ExpressionTestCase) ... ok
test_evaluate_multiply (__main__.ExpressionTestCase) ... ok
test_evaluate_subtract (__main__.ExpressionTestCase) ... ok
test_evaluate_value (__main__.ExpressionTestCase) ... ok
test_to_string_add (__main__.ExpressionTestCase) ... ok
test_to_string_complex_expression (__main__.ExpressionTestCase) ... ok
test_to_string_divide (__main__.ExpressionTestCase) ... ok
test_to_string_multiply (__main__.ExpressionTestCase) ... ok
test_to_string_subtract (__main__.ExpressionTestCase) ... ok
test_to_string_value_expression (__main__.ExpressionTestCase) ... ok
test_find_expression_when_solution_doesnt_exist (__main__.FindEquation) ... ok
test_find_expression_when_solution_exists (__main__.FindEquation) ... ok
test_generate_all_expressions_1_numbers_1_operator (__main__.GenerateExpressions) ... ok
test_generate_all_expr